In [1]:
import numpy as np
from pyvo.dal.adhoc import DatalinkResults

import lsst.afw.display as afwDisplay
from lsst.afw.image import ExposureF
from lsst.rsp import get_tap_service
from lsst.rsp.utils import get_pyvo_auth

from lsst.rsp import get_tap_service

from pyvo.dal.adhoc import DatalinkResults, SodaQuery
import lsst.geom as geom
from astropy import units as u
from astropy.coordinates import Angle
from astropy.time import Time
import lsst.afw.display as afwDisplay
from lsst.afw.image import ExposureF
from lsst.afw.fits import MemFileManager
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd


In [2]:
service = get_tap_service("tap")
assert service is not None

Okay let's pick a SSObject ID sort of at random let's get 4 and we'll use one of them

In [3]:
query = "SELECT * from dp1.SSObject LIMIT 4"


In [4]:


job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()


Job phase is COMPLETED


In [5]:
result = job.fetch_result()
result_df = pd.DataFrame(result)

In [6]:
result_df

,discoverySubmissionDate,numObs,ssObjectId
0,NaN,3,20890932542191434
1,NaN,8,20890945661711695
2,NaN,6,20890950007009348
3,NaN,19,20890950007009349


Let's pick the last one in the table above and get the visit ID for one an observation of it

In [7]:
query = f"SELECT * from dp1.DiaSource where  ssObjectId={result_df.ssObjectId[3]} limit 1"
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()

result = job.fetch_result()
result_df = pd.DataFrame(result)   

Job phase is COMPLETED


In [8]:
result_df

,apFlux,apFluxErr,apFlux_flag,apFlux_flag_apertureTruncated,band,bboxSize,centroid_flag,coord_dec,coord_ra,dec,...,trailDec,trailFlux,trailLength,trailRa,trail_flag_edge,visit,x,xErr,y,yErr
0,917964.0,1142.819946,False,False,i,45,False,7.292676,37.877798,7.292676,...,7.292675,949720.0,1.116525,37.877798,False,2024112700101,1263.061359,0.00371,588.133522,0.005765


In [9]:
result_df.visit[0], result_df.trailRa[0], result_df.trailDec[0], result_df.band[0]

(np.int64(2024112700101),
 np.float64(37.877798130309856),
 np.float64(7.292675331576923),
 np.str_('i'))

Now let's setup up some variables we'll use to grab some cutouts

In [10]:
target_ra = result_df.trailRa[0]
target_dec = result_df.trailDec[0]
visitid = result_df.visit[0]
visitband=result_df.band[0]
circle = (target_ra, target_dec, 0.05)

Now use ivoa Obscore data to get the specific part of the Rubin image we want to make a cutout of

In [11]:

query = """SELECT * FROM ivoa.ObsCore
        WHERE dataproduct_subtype = 'lsst.visit_image' and lsst_visit={} and CONTAINS(POINT('ICRS', {},{}), s_region) = 1
        """.format(visitid, target_ra, target_dec)
print(query)


SELECT * FROM ivoa.ObsCore
        WHERE dataproduct_subtype = 'lsst.visit_image' and lsst_visit=2024112700101 and CONTAINS(POINT('ICRS', 37.877798130309856,7.292675331576923), s_region) = 1
        


In [12]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()
assert job.phase == 'COMPLETED'
results = job.fetch_result().to_table()
print(len(results))

Job phase is COMPLETED
1


In [13]:
results

dataproduct_type,dataproduct_subtype,calib_level,lsst_band,em_min,em_max,lsst_tract,lsst_patch,lsst_filter,lsst_visit,lsst_detector,t_exptime,t_min,t_max,s_ra,s_dec,s_fov,obs_id,obs_collection,o_ucd,facility_name,instrument_name,obs_title,s_region,access_url,access_format,obs_publisher_did,target_name,s_resolution,s_xel1,s_xel2,t_resolution,t_xel,pol_xel,em_xel,em_res_power
,,,,m,m,,,,,,s,d,d,deg,deg,deg,,,,,,,,,,,,arcsec,,,s,,,,
object,object,int32,object,float64,float64,int64,int64,object,int64,int64,float64,float64,float64,float64,float64,float64,object,object,object,object,object,object,object,object,object,object,object,float64,int64,int64,float64,int64,int64,int64,float64
image,lsst.visit_image,2,i,6.936e-07,8.188e-07,--,--,i_06,2024112700101,8,30.0,60642.06064797473,60642.06100028935,37.8761674416365,7.203072917102962,0.31786682440436587,CC_O_20241127_000101,LSST.DP1,phot.flux.density,Rubin:Simonyi,LSSTComCam,visit_image - i - CC_O_20241127_000101-R22_S22 2024-11-28T01:27:19.985016Z,POLYGON ICRS 38.030969 7.243533 37.832635 7.355989 37.721377 7.162634 37.919674 7.050115,https://data.lsst.cloud/api/datalink/links?ID=ivo%3A%2F%2Forg.rubinobs%2Flsst-dp1%3Frepo%3Ddp1%26id%3D019b19cb-58cb-774a-acba-2231d46cabb6,application/x-votable+xml;content=datalink,ivo://org.rubinobs/usdac/lsst-dp1?repo=dp1&id=019b19cb-58cb-774a-acba-2231d46cabb6,Rubin_SV_38_7,--,4072,4000,--,--,--,--,--


Let's grab the image data and make the cutout

In [14]:
datalink_url = results['access_url'][0]
dl_result = DatalinkResults.from_result_url(datalink_url, session=get_pyvo_auth())
image_url = dl_result.getrecord(0).get('access_url')

In [15]:

sq = SodaQuery.from_resource(dl_result,
                             dl_result.get_adhocservice_by_id("cutout-sync-exposure"),
                             session=get_pyvo_auth())


In [16]:


spherePoint = geom.SpherePoint(target_ra*geom.degrees, target_dec*geom.degrees)
Radius = 0.01 * u.deg
sq.circle = (spherePoint.getRa().asDegrees() * u.deg,
             spherePoint.getDec().asDegrees() * u.deg,
             Radius)


In [17]:


cutout_bytes = sq.execute_stream().read()
sq.raise_if_error()
mem = MemFileManager(len(cutout_bytes))
mem.setData(cutout_bytes, len(cutout_bytes))
exposure = ExposureF(mem)


Now display the cutout in the Firefly viewer

In [18]:


afwDisplay.setDefaultBackend("firefly")
display = afwDisplay.Display(frame=1)
display.scale('asinh', 'zscale')
display.image(exposure)
display.setMaskTransparency(100)
display.zoom(2)
plt.show()


Let's do this again but to find another observation where the object is somewhere else so we can show it is moving

In [19]:

query = """SELECT * FROM ivoa.ObsCore
        WHERE dataproduct_subtype = 'lsst.visit_image' and lsst_visit!={} and lsst_band='{}' and CONTAINS(POINT('ICRS', {},{}), s_region) = 1
        """.format(visitid, visitband, target_ra, target_dec)
print(query)

SELECT * FROM ivoa.ObsCore
        WHERE dataproduct_subtype = 'lsst.visit_image' and lsst_visit!=2024112700101 and lsst_band='i' and CONTAINS(POINT('ICRS', 37.877798130309856,7.292675331576923), s_region) = 1
        


We're asking to display this cutout in a second frame in the firefly viewer

In [20]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()
assert job.phase == 'COMPLETED'
results = job.fetch_result().to_table()
print(len(results))

datalink_url = results['access_url'][0]
dl_result = DatalinkResults.from_result_url(datalink_url, session=get_pyvo_auth())
image_url = dl_result.getrecord(0).get('access_url')

sq = SodaQuery.from_resource(dl_result,
                             dl_result.get_adhocservice_by_id("cutout-sync-exposure"),
                             session=get_pyvo_auth())



spherePoint = geom.SpherePoint(target_ra*geom.degrees, target_dec*geom.degrees)
Radius = 0.01 * u.deg
sq.circle = (spherePoint.getRa().asDegrees() * u.deg,
             spherePoint.getDec().asDegrees() * u.deg,
             Radius)


cutout_bytes = sq.execute_stream().read()
sq.raise_if_error()
mem = MemFileManager(len(cutout_bytes))
mem.setData(cutout_bytes, len(cutout_bytes))
exposure = ExposureF(mem)

display = afwDisplay.Display(frame=2)

display.scale('asinh', 'zscale')
display.image(exposure)

display.setMaskTransparency(100)
display.zoom(2)
plt.show()

Job phase is COMPLETED
11


In [21]:
results

dataproduct_type,dataproduct_subtype,calib_level,lsst_band,em_min,em_max,lsst_tract,lsst_patch,lsst_filter,lsst_visit,lsst_detector,t_exptime,t_min,t_max,s_ra,s_dec,s_fov,obs_id,obs_collection,o_ucd,facility_name,instrument_name,obs_title,s_region,access_url,access_format,obs_publisher_did,target_name,s_resolution,s_xel1,s_xel2,t_resolution,t_xel,pol_xel,em_xel,em_res_power
,,,,m,m,,,,,,s,d,d,deg,deg,deg,,,,,,,,,,,,arcsec,,,s,,,,
object,object,int32,object,float64,float64,int64,int64,object,int64,int64,float64,float64,float64,float64,float64,float64,object,object,object,object,object,object,object,object,object,object,object,float64,int64,int64,float64,int64,int64,int64,float64
image,lsst.visit_image,2,i,6.936e-07,8.188e-07,--,--,i_06,2024112600132,5,30.0,60641.07946834481,60641.079820659725,37.92121485024206,7.395226927557233,0.31757934955283157,CC_O_20241126_000132,LSST.DP1,phot.flux.density,Rubin:Simonyi,LSSTComCam,visit_image - i - CC_O_20241126_000132-R22_S12 2024-11-27T01:54:26.064992Z,POLYGON ICRS 38.073057 7.445652 37.867601 7.544835 37.769410 7.344792 37.974767 7.245585,https://data.lsst.cloud/api/datalink/links?ID=ivo%3A%2F%2Forg.rubinobs%2Flsst-dp1%3Frepo%3Ddp1%26id%3D019b1998-7e32-7acf-af0f-36ff5f0c89b6,application/x-votable+xml;content=datalink,ivo://org.rubinobs/usdac/lsst-dp1?repo=dp1&id=019b1998-7e32-7acf-af0f-36ff5f0c89b6,Rubin_SV_38_7,--,4072,4000,--,--,--,--,--
image,lsst.visit_image,2,i,6.936e-07,8.188e-07,--,--,i_06,2024112300238,5,30.0,60638.13935737266,60638.139709525465,37.81551741861292,7.269208763387432,0.31761861943432257,CC_O_20241123_000238,LSST.DP1,phot.flux.density,Rubin:Simonyi,LSSTComCam,visit_image - i - CC_O_20241123_000238-R22_S12 2024-11-24T03:20:40.476998Z,POLYGON ICRS 37.935194 7.374707 37.706980 7.385941 37.695889 7.163717 37.923980 7.152412,https://data.lsst.cloud/api/datalink/links?ID=ivo%3A%2F%2Forg.rubinobs%2Flsst-dp1%3Frepo%3Ddp1%26id%3D019b1997-5d5c-7174-b78d-da21099503c0,application/x-votable+xml;content=datalink,ivo://org.rubinobs/usdac/lsst-dp1?repo=dp1&id=019b1997-5d5c-7174-b78d-da21099503c0,Rubin_SV_38_7,--,4072,4000,--,--,--,--,--
image,lsst.visit_image,2,i,6.936e-07,8.188e-07,--,--,i_06,2024112300239,3,30.0,60638.139971701436,60638.14032395833,37.91923518553408,7.276495649664785,0.31762605379704456,CC_O_20241123_000239,LSST.DP1,phot.flux.density,Rubin:Simonyi,LSSTComCam,visit_image - i - CC_O_20241123_000239-R22_S10 2024-11-24T03:21:33.555004Z,POLYGON ICRS 38.038736 7.382160 37.810507 7.393082 37.799804 7.170751 38.027928 7.159932,https://data.lsst.cloud/api/datalink/links?ID=ivo%3A%2F%2Forg.rubinobs%2Flsst-dp1%3Frepo%3Ddp1%26id%3D019b1976-a3dd-7c6b-a995-31baf7e961f0,application/x-votable+xml;content=datalink,ivo://org.rubinobs/usdac/lsst-dp1?repo=dp1&id=019b1976-a3dd-7c6b-a995-31baf7e961f0,Rubin_SV_38_7,--,4072,4000,--,--,--,--,--
image,lsst.visit_image,2,i,6.936e-07,8.188e-07,--,--,i_06,2024112300243,3,30.0,60638.14236084512,60638.142713125,37.95169413849998,7.349572526538416,0.31763289588097016,CC_O_20241123_000243,LSST.DP1,phot.flux.density,Rubin:Simonyi,LSSTComCam,visit_image - i - CC_O_20241123_000243-R22_S10 2024-11-24T03:24:59.977018Z,POLYGON ICRS 38.056612 7.469534 37.828817 7.451409 37.846843 7.229548 38.074533 7.247748,https://data.lsst.cloud/api/datalink/links?ID=ivo%3A%2F%2Forg.rubinobs%2Flsst-dp1%3Frepo%3Ddp1%26id%3D019b1976-c50a-7dd0-aeab-65b52eaadcf4,application/x-votable+xml;content=datalink,ivo://org.rubinobs/usdac/lsst-dp1?repo=dp1&id=019b1976-c50a-7dd0-aeab-65b52eaadcf4,Rubin_SV_38_7,--,4072,4000,--,--,--,--,--
image,lsst.visit_image,2,i,6.936e-07,8.188e-07,--,--,i_06,2024112300247,3,30.0,60638.14480740764,60638.145159699074,37.97370150739136,7.389066847805279,0.31762174807245463,CC_O_20241123_000247,LSST.DP1,phot.flux.density,Rubin:Simonyi,LSSTComCam,visit_image - i - CC_O_20241123_000247-R22_S10 2024-11-24T03:28:31.360020Z,POLYGON ICRS 38.078626 7.509023 37.850807 7.490906 37.868852 7.269051 38.096545 7.287229,https://data.lsst.cloud/api/datalink/links?